In [3]:
import pandas as pd

from augment import generate_sentences_by_shuffle_within_segments, generate_sentences_by_replace_mention, generate_sentences_by_replace_token, generate_sentences_by_synonym_replacement
from tqdm import tqdm
import os


import argparse, json, logging, numpy, os, random, sys, torch

from data import ConllCorpus
from train import train, final_test
from models import TransformerEncoder, LinearCRF, MLP
from augment import get_category2mentions, get_label2tokens


In [4]:
data_path = 'E:/打工/竞赛/GAIIC/LDA分析/聚类切分结果txt(训练集格式)/'
data_list = os.listdir(data_path)
data_list

['0_train.txt',
 '10_train.txt',
 '11_train.txt',
 '12_train.txt',
 '13_train.txt',
 '14_train.txt',
 '1_train.txt',
 '2_train.txt',
 '3_train.txt',
 '4_train.txt',
 '5_train.txt',
 '6_train.txt',
 '7_train.txt',
 '8_train.txt',
 '9_train.txt']

In [6]:
#augmentation_methods = ["MR", "LwTR", "SiS", "SR"]
augmentation_methods = ["SR"]

save_path = 'E:/打工/竞赛/GAIIC/LDA分析/按聚类结果增强的数据/'
for file in tqdm(data_list):
    train_txt = data_path+file
    corpus = ConllCorpus('development', train_txt, './data/dev.txt', './data/test.txt')
    tag_dict = corpus.build_tag_dict("gold")

    category2mentions = get_category2mentions(corpus.train)
    label2tokens = get_label2tokens(corpus.train, p_power=1)

    augmented_sentences = []
    for s in corpus.train:
        if "MR" in augmentation_methods:
                augmented_sentences += generate_sentences_by_replace_mention(sentence=s,
                                                                             category2mentions=category2mentions,
                                                                             replace_ratio=0.3,
                                                                             num_generated_samples=1)
        if "LwTR" in augmentation_methods:
                augmented_sentences += generate_sentences_by_replace_token(sentence=s,
                                                                           label2tokens=label2tokens,
                                                                           replace_ratio=0.3,
                                                                           num_generated_samples=1)
        if "SiS" in augmentation_methods:
                augmented_sentences += generate_sentences_by_shuffle_within_segments(sentence=s,
                                                                                     replace_ratio=0.3,
                                                                                     num_generated_samples=1)
        if "SR" in augmentation_methods:
                augmented_sentences += generate_sentences_by_synonym_replacement(sentence=s,
                                                                                 replace_ratio=0.3,
                                                                                 num_generated_samples=1)


    with open(save_path+'aug_'+file, 'w', encoding='utf-8') as f:
        for s in augmented_sentences:
            label = []
            for t in s:
                label.append(t.get_label("gold"))
            token = s.tokens
            for name, target in zip(token, label):
                f.write(str(name))
                f.write(' ')
                f.write(target)
                f.write('\n')

            f.write('\n')

100%|██████████| 15/15 [00:23<00:00,  1.59s/it]


100%|██████████| 36000/36000 [00:02<00:00, 14254.91it/s]


'手'